In [ ]:
# !unzip ./Traffic_signs.zip

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
# import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from keras.models import Sequential, load_model

In [ ]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = np.array(image.resize((64,64)))
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")


In [ ]:
#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

(39209, 64, 64, 3) (39209,)
(31367, 64, 64, 3) (7842, 64, 64, 3) (31367,) (7842,)


In [ ]:
def my_model():
    # inputs = layers.InputLayer(input_shape=(64, 64, 3))
    inputs = keras.Input(shape=(64, 64, 3))
    x = layers.Conv2D(
        filters=32,
        kernel_size=3,
        padding="same",
        kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPool2D(pool_size=(2,2))(x)
    x = layers.Conv2D(64,3)(x)
    # x = layers.Conv2D(64,3,kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPool2D(pool_size=(2,2))(x)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    # x = layers.Conv2D(64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
    x = layers.Conv2D(128,3, activation="relu")(x)
    x = layers.MaxPool2D(pool_size=(2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64,activation='relu')(x)
    outputs = layers.Dense(43, activation="softmax")(x)
    model = keras.Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.0007

model = my_model()
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10)
model.evaluate(X_test,y_test)

Epoch 1/10
981/981 [==============================] - 39s 7ms/step - loss: 1.4069 - accuracy: 0.5921
Epoch 2/10
981/981 [==============================] - 7s 7ms/step - loss: 0.3331 - accuracy: 0.8944
Epoch 3/10
981/981 [==============================] - 7s 7ms/step - loss: 0.1824 - accuracy: 0.9436
Epoch 4/10
981/981 [==============================] - 7s 7ms/step - loss: 0.1248 - accuracy: 0.9624
Epoch 5/10
981/981 [==============================] - 7s 7ms/step - loss: 0.0928 - accuracy: 0.9731
Epoch 6/10
981/981 [==============================] - 7s 7ms/step - loss: 0.0756 - accuracy: 0.9780
Epoch 7/10
981/981 [==============================] - 7s 7ms/step - loss: 0.0640 - accuracy: 0.9826
Epoch 8/10
981/981 [==============================] - 7s 7ms/step - loss: 0.0555 - accuracy: 0.9849
Epoch 9/10
981/981 [==============================] - 7s 7ms/step - loss: 0.0491 - accuracy: 0.9863
Epoch 10/10
246/246 [==============================] - 1s 4ms/step - loss: 0.0200 - accuracy: 0.996

[0.019969629123806953, 0.9965569972991943]

In [ ]:
model.save('traffic_signs_classifier/')

INFO:tensorflow:Assets written to: traffic_signs_classifier/assets


In [ ]:
# !zip -r "model.zip" traffic_signs_classifier/ 

In [ ]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((64,64))
    data.append(np.array(image))

X_test=np.array(data)
pred = [np.argmax(p) for p in model.predict(X_test)] 
pred = np.array(pred)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

0.9771971496437054
